# Darbas su SQL Python aplinkoje

In [3]:
# importuojame tvarkylę sqlite3
import sqlite3

In [4]:
# sukūriame prisijungimo objektą connector. Jeigu prieš tai neturėjome failo zmones.db, jis sukuriamas automatiškai.
connector = sqlite3.connect('data/zmones.db')
# jo pagrindu sukūriam kursoriaus objektą cursor. Su jo metodais vykdomos SQL užklausos.
cursor = connector.cursor()
# query yra mūsų SQL užklausa.
query = '''
CREATE TABLE IF NOT EXISTS draugai (
    first_name VARCHAR(50),
    last_name VARCHAR(100),
    email VARCHAR(255)
);
'''
# į kursoriaus metodo excecute parametrus dedame savo užklausą vykdymui.
cursor.execute(query)
# connector.commit() - išsaugo pakeitimus duomenų bazėje.
connector.commit()
# connector.close() - uždarome atidarytą prisijungimą.
connector.close()

# Jeigu dar kartą leistumėm tą pačią programą, gautumėm klaidą, kadangi tokia lentelė jau sukurta. 
# Todėl programose kuriant lenteles, pravartu papildyti sąlyga:

#CREATE TABLE IF NOT EXISTS lentelė (
#    ....

In [7]:
connector = sqlite3.connect('data/zmones.db')

## Įrašymas

In [9]:
# duomenų įterpimas
query = '''
INSERT INTO draugai
(first_name, last_name, email)
VALUES
("Smagus", "Gerulis", "gerulis@gmail.com")
'''
# context manager'is atidaro DB, atlieka reikiamus commit ir uždaro
connector = sqlite3.connect("data/zmones.db")
with connector:
    connector.cursor().execute(query)

In [16]:
query = '''
INSERT INTO draugai
(first_name, last_name, email)
VALUES
("Geras", "Smagulis", "gerulis@gmail.com")
'''
connector = sqlite3.connect("data/zmones.db")
with connector:
    connector.cursor().execute(query)

In [24]:
query = '''
INSERT INTO draugai
(first_name, last_name, email)
VALUES
("Geras", "Programuotojas", "coder@gmail.com")
'''
connector = sqlite3.connect("data/zmones.db")
with connector:
    connector.cursor().execute(query)

## .fetchone()

In [26]:
# skaitome duomenų bazę (po vieną)
with connector:
    # kursorių reikia pasiimti į kintamąjį, nes jame nugula rezultato generatorius
    cursor = connector.cursor()
    cursor.execute("SELECT * FROM draugai;")
    print(cursor.fetchone())
    print(cursor.fetchone()) # antrakart spausdinant ima antra irasa
    print(cursor.fetchone())
    print(cursor.fetchone()) # None, nes nebera ten daugiau eiluciu, tik trys

('Smagus', 'Gerulis', 'gerulis@gmail.com')
('Geras', 'Smagulis', 'gerulis@gmail.com')
('Geras', 'Programuotojas', 'coder@gmail.com')
None


## .fetchall()

Jeigu rezultatų daugiau, negu vienas, .fetchone() mums spausdina pirmą rezultatą. Norint gauti juos visus, turėtumem naudoti .fetchall():

In [18]:
# nuskaitome visus duomenis į sąrašą su .fetchall()
with connector:
    cursor = connector.cursor()
    cursor.execute("SELECT * FROM draugai;")
    print(cursor.fetchall())

[('Smagus', 'Gerulis', 'gerulis@gmail.com'), ('Geras', 'Smagulis', 'gerulis@gmail.com')]


## Įrašų keitimas ir trynimas

Įrašų atnaujinimui ir trynimui nereikia naudoti jokių specifinių metodų.

In [15]:
# taisom situaciją
with connector:
    cursor = connector.cursor()
    # slaptas sqlite'o irasas rowid
    cursor.execute("DELETE FROM draugai WHERE rowid > 1;")
    cursor.execute("SELECT * FROM draugai;")
    print(cursor.fetchall())

[('Smagus', 'Gerulis', 'gerulis@gmail.com')]


In [31]:
from pprint import pprint
# nuskaitome visus duomenis į sąrašą su .fetchall()
with connector:
    cursor = connector.cursor()
    cursor.execute("SELECT * FROM draugai;")
    pprint(cursor.fetchall()) # atkreipt dėmesį, kad čia PPRINT

[('Smagus', 'Gerulis', 'gerulis@gmail.com'),
 ('Geras', 'Smagulis', 'gerulis@gmail.com'),
 ('Žalias', 'Programuotojas', 'coder@gmail.com')]


In [27]:
# įrašų atnaujinimas
with connector:
    cursor = connector.cursor()
    cursor.execute('UPDATE draugai SET first_name="Žalias" WHERE last_name="Programuotojas";')
    cursor.execute("SELECT * FROM draugai;")
    print(cursor.fetchall())

[('Smagus', 'Gerulis', 'gerulis@gmail.com'), ('Geras', 'Smagulis', 'gerulis@gmail.com'), ('Žalias', 'Programuotojas', 'coder@gmail.com')]


## excecutemany()

In [8]:
nauji_draugai = [
    ('Naujas', 'Draugelis', 'naujas@draugelis.lt'),
    ('Grazi', 'Mergaite', 'grazuole@gmail.com'),
    ('Senas', 'Diedulis', 'diedux@yahoo.com')
]

with connector:
    cursor = connector.cursor()
    cursor.executemany('INSERT INTO draugai VALUES (?,?,?)', (nauji_draugai))

## Rowid

In [9]:
with connector:
    cursor = connector.cursor()
    cursor.execute('SELECT rowid, * FROM draugai WHERE rowid > 5')
    draugai = cursor.fetchall()
    for draugas in draugai:
        print(f'- {draugas}')

- (6, 'Blogas', 'Hakeris', 'haxxxxx@dark.net')
- (7, 'Naujas', 'Draugelis', 'naujas@draugelis.lt')
- (8, 'Grazi', 'Mergaite', 'grazuole@gmail.com')
- (9, 'Senas', 'Diedulis', 'diedux@yahoo.com')


In [11]:
ids = (1, 3, 5, 7)
with connector:
    cursor = connector.cursor()
    cursor.execute('SELECT rowid, * FROM draugai WHERE rowid IN (?, ?, ?, ?)', ids)
    draugai = cursor.fetchall()
    for draugas in draugai:
        print(f'- {draugas}')

- (1, 'Smagus', 'Gerulis', 'gerulis@gmail.com')
- (3, 'Žalias', 'Programuotojas', 'coder@gmail.com')
- (5, 'Grybas', 'Fungauskas', 'grybas@fungus.net')
- (7, 'Naujas', 'Draugelis', 'naujas@draugelis.lt')
